In [1]:
import os
import sys
import PATH
import numpy as np
import pandas as pd
import utils

import seaborn as sns
from matplotlib import pyplot as plt

from importlib import reload
import analysis_for_MTL as A_MTL
import ribo_public.parse_ribosome as Pribo
from sklearn import preprocessing

from models import Backbone
from models import reader
from models import train_val
from models import Cross_stitch
from models.popen import Auto_popen

/home/wergillius/Project/UTR_VAE


# <font color='skyblue'>LOADING PREDICTOR MODEL: 
- `Backbone/RL_regressor_MTL/split_two_csv_deep/slr_l1_scaled`

In [6]:
NBT_dir = "/data/users/wergillius/UTR_VAE/Ex_data"
log_dir = "/home/wergillius/Project/UTR_VAE/log/Backbone/RL_regressor_MTL/fast_distribution"
slr_l1 = os.path.join(log_dir,"student_model.ini")

POPEN = Auto_popen(slr_l1)

In [3]:
# redirect the log path
POPEN.vae_log_path

'/home/wergillius/Project/UTR_VAE/log/Backbone/RL_regressor_MTL/fast_distribution/student_model.log'

In [4]:
train_df = pd.read_csv(POPEN.split_like_paper[0],index_col=0)
test_df = pd.read_csv(POPEN.split_like_paper[1],index_col=0)

print(train_df.shape)
print(test_df.shape)

utr_M = test_df.utr.to_numpy()
rl_M = test_df.rl.to_numpy()


TypeError: 'bool' object is not subscriptable

In [8]:
round_random_index = np.load(os.path.join(NBT_dir,'random_index.M.npy'))

In [74]:
np.save(os.path.join(NBT_dir,'random_index.M'),round_random_index)

In [ ]:
plt.figure(figsize=(8,4),dpi=600)

for i, round_index in enumerate(round_random_index):
    rl_sampled = np.stack([val_set[idx][1] for idx in round_index])

    sns.kdeplot(rl_sampled,label="round %s"%i,alpha=0.6);
plt.legend();

# train Backone: <font color='orange'> RL_regressor 

In [9]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset,DataLoader,random_split

<font size=4 color='skyblue'>**define model**

In [10]:
Model_Class = POPEN.Model_Class  # DL_models.LSTM_AE
model = Model_Class(*POPEN.model_args).cuda(POPEN.cuda_id)

<font size=4 color='skyblue'>**define logger and optimizer**

In [11]:
logger = utils.setup_logs(POPEN.vae_log_path)

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                            lr=POPEN.lr,
                            betas=(0.9, 0.98), 
                            eps=1e-09, 
                            weight_decay=POPEN.l2)

<font size=4 color='skyblue'>**define dataset**

In [12]:
dataset_func = lambda x : reader.MTL_enc_dataset(
    DF=x,pad_to=POPEN.pad_to,aux_columns=POPEN.aux_task_columns,
    input_col=POPEN.other_input_columns)

class sub_round_dataset(Dataset):
    def __init__(self,entire_val_set,round_x_index):
        self.all_data = entire_val_set
        self.sampled_index = round_x_index
        
    def __len__(self):
        return len(self.sampled_index)
        
    def __getitem__(self,index):
        i = self.sampled_index[index]
        return self.all_data[i]

def get_two_set(dataset,ratio=POPEN.train_test_ratio,seed = 42):
    total_len = len(dataset)
    lengths = [int(total_len*sub_ratio) for sub_ratio in ratio[:-1]]
    lengths.append(total_len-sum(lengths))         # make sure the sum of length is the total len

    train_set,val_set = random_split(dataset,lengths,generator=torch.Generator().manual_seed(seed))
    return train_set,val_set

def DF_to_dataloader(full_df,POPEN):
    df_ls = [full_df]

    dataset_func = lambda x : reader.MTL_enc_dataset(DF=x,pad_to=POPEN.pad_to,
                                aux_columns=POPEN.aux_task_columns,input_col=POPEN.other_input_columns)

    loader_ls = reader.get_splited_dataloader(dataset_func,df_ls,ratio=POPEN.train_test_ratio,
                                    batch_size=POPEN.batch_size,num_workers=4,seed=42) #
    return loader_ls

In [13]:
DF = pd.read_csv(POPEN.csv_path)
dataset = dataset_func(DF)
# the train_set and val_set were used to train the teacher model
train_set, val_set = get_two_set(dataset)  

# val_df is the entire dataset used to train small_model
val_df = DF.iloc[val_set.indices]

/home/wergillius/.conda/envs/pytorch/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (28,37,82,83,84,85,86,87,88,89) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [23]:

train_set, val_set = get_two_set(dataset)  

# val_df is the entire dataset used to train small_model
train_df = DF.iloc[train_set.indices]

In [24]:
with_col = [col for col in train_df.columns if "with_" in col]

In [25]:
for col in with_col:
    train_df.loc[:,col]= train_df.loc[:,col].apply(lambda x: int(x))

/home/wergillius/.conda/envs/pytorch/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [27]:
NBT_dir = "/data/users/wergillius/UTR_VAE/Ex_data"
train_df.to_csv(os.path.join(NBT_dir,'big_model_dataset.csv'),index=False)

```python
NBT_dir = "/data/users/wergillius/UTR_VAE/Ex_data"
val_df.to_csv(os.path.join(NBT_dir,'small_model_dataset.csv'),index=False)
```

In [14]:
log_dir = "/home/wergillius/Project/UTR_VAE/log/Backbone/RL_regressor_MTL/fast_distribution"
slr_l1 = os.path.join(log_dir,"student_model.ini")
POPEN = Auto_popen(slr_l1)

In [22]:
round0_df = pd.DataFrame([])
POPEN.vae_pth_path = "/data/users/wergillius/UTR_VAE/pth/RL_regressor_MTL/fast_distribution/student_round-1-model_best.pth"
for round_x in range(10):

    round0_df = round0_df.append(val_df.iloc[round_random_index[round_x]])
    round0_trainloader,round0_valloader,_ = DF_to_dataloader(round0_df,POPEN)
    
    Model_Class = POPEN.Model_Class  # DL_models.LSTM_AE
    model = Model_Class(*POPEN.model_args).cuda(POPEN.cuda_id)
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                            lr=POPEN.lr,
                            betas=(0.9, 0.98), 
                            eps=1e-09, 
                            weight_decay=POPEN.l2)
    
    logger.info(" ROUND : %s"%round_x)
    
    for epoch in range(10):
        logger.info("\n===============================|    epoch {}   |===============================\n".format(epoch))
        train_val.train(dataloader=round0_trainloader,
                        model=model,optimizer=optimizer,
                        popen=POPEN,epoch=epoch)

    val_total_loss,val_avg_acc = train_val.validate(round0_valloader,model,popen=POPEN,epoch=epoch)

    POPEN.vae_pth_path = POPEN.vae_pth_path.replace("_round%s"%(round_x-1),"_round%s"%round_x)

    utils.snapshot(POPEN.vae_pth_path, {
                            'epoch': epoch + 1,
                            'validation_acc': val_avg_acc,
                            'state_dict': model.state_dict(),
                            'validation_loss': val_total_loss,
                            'optimizer': optimizer.state_dict(),
                        })

 ROUND : 0
 ROUND : 0

===============================|    epoch 0   |===============================


===============================|    epoch 0   |===============================

    0 /   375 (0.0%):	 Total:1.7028717	 Acc:0.1250000	 Mean_Total:1.7028717
    0 /   375 (0.0%):	 Total:1.7028717	 Acc:0.1250000	 Mean_Total:1.7028717
   75 /   375 (20.0%):	 Total:2.1704628	 Acc:0.2500000	 Mean_Total:1.2835519
   75 /   375 (20.0%):	 Total:2.1704628	 Acc:0.2500000	 Mean_Total:1.2835519
  150 /   375 (40.0%):	 Total:0.6923342	 Acc:0.1875000	 Mean_Total:1.1992914
  150 /   375 (40.0%):	 Total:0.6923342	 Acc:0.1875000	 Mean_Total:1.1992914
  225 /   375 (60.0%):	 Total:1.6817527	 Acc:0.1875000	 Mean_Total:1.1732987
  225 /   375 (60.0%):	 Total:1.6817527	 Acc:0.1875000	 Mean_Total:1.1732987
  300 /   375 (80.0%):	 Total:0.9007486	 Acc:0.2812500	 Mean_Total:1.1491887
  300 /   375 (80.0%):	 Total:0.9007486	 Acc:0.2812500	 Mean_Total:1.1491887

===============================|    epoch 1   |

# Train <font color='orange'> Cross stitch

In [ ]:
a5b5_config = "/home/wergillius/Project/UTR_VAE/log/CrossStitch/CrossStitch_Model_MTL/a5b5/a5b5_g2_11.ini"
a5b5_popen = Auto_popen(a5b5_config)
a5b5_popen.vae_log_path = "/home/wergillius/Project/UTR_VAE/log/CrossStitch/CrossStitch_Model_MTL/fast_distibution/a5b5_g2_11.log"
logger = utils.setup_logs(a5b5_popen.vae_log_path)

recon_config = "/home/wergillius/Project/UTR_VAE/log/Backbone/Reconstruction_MTL/pretrain_with_um2_p1_design/kld1e_5_l1.ini"
recon_popen = Auto_popen(recon_config)
recon_model = recon_popen.Model_Class(*recon_popen.model_args)
utils.load_model(recon_popen,recon_model,logger)

motif_config = "/home/wergillius/Project/UTR_VAE/log/Backbone/Motif_detection_MTL/pretrain_with_unm2_p1_design/eight_motif_test.ini"
motif_popen = Auto_popen(motif_config)

a5b5_popen.tasks

log_dir = "/home/wergillius/Project/UTR_VAE/log/Backbone/RL_regressor_MTL/fast_distribution"
slr_l1 = os.path.join(log_dir,"student_model.ini")
POPEN = Auto_popen(slr_l1)